In [1]:
import time # 딜레이를 사용하기 위한 라이브러리
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
# 키보드를 제어하는 라이브러리
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
# 웹사이트의 모든 자원 로드를 기다리기 위해 암묵적으로 5초 대기. 그 전에 로딩이 끝나면 5초가 안 됐어도 다음 코드로 넘어감
driver.implicitly_wait(5)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/jason/.wdm/drivers/chromedriver/mac64/99.0.4844.51]
/var/folders/81/99qz1j0s6_96dwmd218v9sph0000gn/T/ipykernel_60077/2634969298.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False
def remove_tags(element):
    result = ''
    try:
        result = re.sub('<.+?>', '', str(element), 0).strip()
        return result
    except NoSuchElementException:
            return ''

In [16]:
keyword=input("검색어를 입력하세요.")
baseUrl='https://www.daangn.com'
url = baseUrl+'/search/'+keyword
driver.get(url)
html = ''
result_flea = True
try:
    check_exists_by_xpath('//*[@id="flea-market-wrap"]')
    result_flea = True
except:
    result_flea = False
    
if(result_flea == True) :
    count = 0 
    while (check_exists_by_xpath('//*[@id="result"]/div[1]/div[2]') == True) :
        try:
            driver.find_element_by_xpath('//*[@id="result"]/div[1]/div[2]').click()
            time.sleep(0.5)
        except:
            html = bs(driver.page_source, 'html.parser')
            break
            
html = bs(driver.page_source, 'html.parser')

#가끔 클릭이되는 오류가 있음

검색어를 입력하세요.레페토


/var/folders/81/99qz1j0s6_96dwmd218v9sph0000gn/T/ipykernel_5672/679012471.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(xpath)
/var/folders/81/99qz1j0s6_96dwmd218v9sph0000gn/T/ipykernel_5672/184082566.py:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="result"]/div[1]/div[2]').click()


MaxRetryError: HTTPConnectionPool(host='localhost', port=61498): Max retries exceeded with url: /session/5555792e5de27af6b0f2cefa27d20725/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc68534fdf0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [15]:
html

<html lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
<link href="https://www.daangn.com/search/%EC%9A%B8%EB%A6%89%EB%8F%84%EC%9E%90%EC%A0%84%EA%B1%B0" rel="canonical"/>
<title>울릉도자전거 중고거래 | 당신 근처의 당근마켓</title>
<meta content="자전거, 울릉" name="keywords"/>
<link href="당근마켓" rel="author"/>
<meta content="https://www.daangn.com/search/%EC%9A%B8%EB%A6%89%EB%8F%84%EC%9E%90%EC%A0%84%EA%B1%B0" property="og:url"/>
<meta content="울릉도자전거 중고거래 | 당신 근처의 당근마켓" property="og:title"/>
<meta content="당근마켓" property="og:site_name"/>
<meta content="https://www.daangn.com/images/meta/home/daangn.png" property="og:image"/>
<meta content="article" property="og:type"/>
<meta content="ko_KR" property="og:locale"/>
<meta content="1463621440622064" property="fb:app_id"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@danggeunmarket" name="t

In [55]:
products = html.select('#flea-market-wrap > article')

prod_list = []

for product in products :
    prod_list.append(product)

datas = []
for prod in prod_list :
    container = dict()
    try:
        container['title'] = remove_tags(prod.find('img')['alt'])
        container['img'] = prod.find('img')['src']
        container['content'] = remove_tags(prod.find('span', {'class' : 'article-content'}))
        container['region'] = remove_tags(prod.find('p', {'class' : 'article-region-name'}))
        container['price'] = remove_tags(prod.find('p', {'class' : 'article-price'}))
        container['link'] = baseUrl+remove_tags(prod.find('a', {'class' : 'flea-market-article-link'})['href'])
    except :
        print('error', prod)
    datas.append(container)

# print(datas)

In [56]:
df = pd.DataFrame(datas)
# csv파일로 저장
df.to_csv('./'+keyword+'.csv', index=False)
# 데이터 확인
df

,title,img,content,region,price,link
0,repetto 레페토 zizi shoes,https://dnvefa72aowie.cloudfront.net/origin/ar...,"레페토 시그니처 모델 zizi입니다. 38size\n100% 염소가죽, 2cm굽\n...",서울 종로구 부암동,"186,000원",https://www.daangn.com/articles/347606558
1,REPETTO 레페토 클레멍스 블랙 구두 230(36),https://dnvefa72aowie.cloudfront.net/origin/ar...,"정가 398,000원\n\n2021. 12. 31 신세계 S.I.Village에서 ...",서울 영등포구 영등포동1가,"240,000원",https://www.daangn.com/articles/347827123
2,(미착용 새상품) Repetto 레페토 샌들 41 편한 여자샌들 여자신발,https://dnvefa72aowie.cloudfront.net/origin/ar...,(미착용 새상품) Repetto 레페토 샌들 41 편한 여자샌들\n\n선물받은 새제...,서울 강남구 역삼동,"150,000원",https://www.daangn.com/articles/373830378
3,(미착용 새상품) Repetto 레페토 샌들 41 편한 여자샌들 여자신발,https://dnvefa72aowie.cloudfront.net/origin/ar...,(미착용 새상품) Repetto 레페토 샌들 41 편한 여자샌들\n\n선물받은 새제...,서울 송파구 잠실3동,"150,000원",https://www.daangn.com/articles/373839333
4,레페토 지지 단화 구두 repetto 245 39,https://dnvefa72aowie.cloudfront.net/origin/ar...,레페토 지지 repetto 245 39,서울 용산구 이태원제1동,"15,000원",https://www.daangn.com/articles/370219239
...,...,...,...,...,...,...
9133,레페토×꼼데가르송 콜라보 슈즈 36사이즈,https://dnvefa72aowie.cloudfront.net/origin/ar...,래페토 꼼데가르송 콜라보 슈즈입니다.\n원래 레페토 37사이즈 신는데 36사이즈 단...,충북 청주시 서원구 산남동,"10,000원",https://www.daangn.com/articles/26558909
9134,레페토 까미유 37사이즈,https://dnvefa72aowie.cloudfront.net/origin/ar...,신라면세구입\n2번정도 신고 보관만\n\n상태 좋아요,경기도 안산시 상록구 성포동,"150,000원",https://www.daangn.com/articles/25917960
9135,"[새상품]슈펜 레페토 리본단화 4cm 245사이즈,네이비",https://dnvefa72aowie.cloudfront.net/origin/ar...,부드러운 합피여서 앞에 까짐없고 \n쿠셔닝이 도톰해서 착화감 편해요 \n\n✔️굽:...,서울 강남구 신사동,"8,000원",https://www.daangn.com/articles/26078181
9136,레페토 정품,https://dnvefa72aowie.cloudfront.net/origin/ar...,"이태리에서 구매했어요\n245,250가능합니다",부산 연제구 연산제4동,"35,000원",https://www.daangn.com/articles/24847778
